# what is the approximate depth of a Decision Tree trained(without restrictions)on a training set with one million instances?

# Is a node's Gine impurity generally lower or greater than its parent's?Is it generally low/greater,or always lower/greater?

# If a Decision Tree is overfitting the training set,is it a good idea to try decreasing max_depth?

# If a Decision Tree is underfitting the training set,is it a good idea to try scaling the input features?

# If it takes one hour to train a Decision Tree on a training set containing 1 million instances,roughly how much time will it take to train another Decision Tree on a training set containing 10 million instances?

# If your taining set contains 100,000 instances,will setting presort=True speed up training?